In [ ]:
# | default_exp task

# Task

> Tasks to interact with the Archivematica REST API


In [ ]:
# | hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# | export
from archivematica_tools.api import ArchivematicaAPIClient
from archivematica_tools.aws import AwsClient
from typing import Optional

In [ ]:
# | export
class Task:

    def __init__(self, dashboard_url: str, dashboard_username: str, dashboard_api_key: str,
                 storage_service_url: str, storage_service_username: str, storage_service_password: str,
                 aws_access_key_id: str, aws_secret_access_key: str, aws_region: str):
        """
        Constructor for the Task class. Initializes the Matica API client and the AWS S3 client.

        Args:
            dashboard_url (str): URL for the Matica dashboard.
            dashboard_username (str): Username for the Matica dashboard.
            dashboard_api_key (str): API key for the Matica dashboard.
            storage_service_url (str): URL for the storage service.
            storage_service_username (str): Username for the storage service.
            storage_service_password (str): Password for the storage service.
            aws_access_key_id (str): AWS access key ID.
            aws_secret_access_key (str): AWS secret access key.
            aws_region (str): AWS region.
        """
        self.matica_client = ArchivematicaAPIClient(dashboard_url, dashboard_username, dashboard_api_key,
                                             storage_service_url, storage_service_username, storage_service_password)
        self.aws_client = AwsClient(aws_access_key_id, aws_secret_access_key, aws_region)

    @staticmethod
    def ensure_slash_suffix(s: str) -> str:
        """
        Ensures the specified string ends with a slash, adding one if it does not.

        Args:
            s (str): The string to process.

        Returns:
            str: The string guaranteed to end with a slash.
        """
        return s if s.endswith('/') else f'{s}/'

    @staticmethod
    def main(dashboard_url: str, dashboard_username: str, dashboard_api_key: str,
             storage_service_url: str, storage_service_username: str, storage_service_password: str,
             aws_access_key_id: str, aws_secret_access_key: str, task_id: str, transfer_type: str, 
             transfer_name: str, file_path: str, location_uuid: str, processing_config: str, 
             bucket_name: str, transfer_source_prefix: str, aws_region: str) -> Optional[str]:
        """
        Executes the main process. Uploads a ZIP file to S3, then triggers processing in Matica.

        Args:
            The following arguments are similar to those in the __init__ method.
            task_id (str): Task ID.
            transfer_type (str): Transfer type.
            transfer_name (str): Transfer name.
            file_path (str): File path.
            location_uuid (str): Location UUID.
            processing_config (str): Processing configuration.
            bucket_name (str): Bucket name.
            transfer_source_prefix (str): Transfer source prefix.
            aws_region (str): AWS region.

        Returns:
            Optional[str]: The URL of the processed file if successful, None otherwise.
        """
        
        task = Task(dashboard_url, 
        dashboard_username,
        dashboard_api_key,
        storage_service_url,
        storage_service_username,
        storage_service_password,
        aws_access_key_id,
        aws_secret_access_key,
        aws_region
        )

        matica_client = task.matica_client

        transfer_source_prefix = Task.ensure_slash_suffix(transfer_source_prefix)
        s3_path = f"{transfer_source_prefix}{task_id}"

        task.aws_client.upload_to_s3_zip(file_path, s3_path, bucket_name)

        transfer_UUID = task.matica_client.v2beta_package(transfer_type, "", location_uuid, "/" + s3_path, transfer_name, processing_config)

        sip_UUID = matica_client.check_transfer_status(transfer_UUID)
        ingest_UUID = matica_client.ingest(sip_UUID)

        url = f"https://s3.{aws_region}.amazonaws.com/{bucket_name}/{task.matica_client.get_current_full_path(ingest_UUID)}"

        return url

In [ ]:
show_doc(Task)

---

[source](https://github.com/nakamura196/archivematica_tools/blob/main/archivematica_tools/task.py#L12){target="_blank" style="float:right; font-size:smaller"}

### Task

>      Task (dashboard_url:str, dashboard_username:str, dashboard_api_key:str,
>            storage_service_url:str, storage_service_username:str,
>            storage_service_password:str, aws_access_key_id:str,
>            aws_secret_access_key:str, aws_region:str)

Constructor for the Task class. Initializes the Matica API client and the AWS S3 client.

Args:
    dashboard_url (str): URL for the Matica dashboard.
    dashboard_username (str): Username for the Matica dashboard.
    dashboard_api_key (str): API key for the Matica dashboard.
    storage_service_url (str): URL for the storage service.
    storage_service_username (str): Username for the storage service.
    storage_service_password (str): Password for the storage service.
    aws_access_key_id (str): AWS access key ID.
    aws_secret_access_key (str): AWS secret access key.
    aws_region (str): AWS region.

# Main

In [ ]:
show_doc(Task.main)

---

[source](https://github.com/nakamura196/archivematica_tools/blob/main/archivematica_tools/task.py#L47){target="_blank" style="float:right; font-size:smaller"}

### Task.main

>      Task.main (dashboard_url:str, dashboard_username:str,
>                 dashboard_api_key:str, storage_service_url:str,
>                 storage_service_username:str, storage_service_password:str,
>                 aws_access_key_id:str, aws_secret_access_key:str, task_id:str,
>                 transfer_type:str, transfer_name:str, file_path:str,
>                 location_uuid:str, processing_config:str, bucket_name:str,
>                 transfer_source_prefix:str, aws_region:str)

Executes the main process. Uploads a ZIP file to S3, then triggers processing in Matica.

Args:
    The following arguments are similar to those in the __init__ method.
    task_id (str): Task ID.
    transfer_type (str): Transfer type.
    transfer_name (str): Transfer name.
    file_path (str): File path.
    location_uuid (str): Location UUID.
    processing_config (str): Processing configuration.
    bucket_name (str): Bucket name.
    transfer_source_prefix (str): Transfer source prefix.
    aws_region (str): AWS region.

Returns:
    Optional[str]: The URL of the processed file if successful, None otherwise.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()